<h1 style="font-size:50px;" align="center">Inferencia Bayesiana usando el Motor de Inferencia MIB</h1>

<h1 style="font-size:30px;" align="center">Quien escribió este cuento</h1>

In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import glob
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

from collections import Counter #regresa un diccionario con conteos
from collections import OrderedDict #diccionarios ordenados
from itertools import product,islice   #calcula producto cartesiano y saca una rebanada de diccionario


import mib as mb
import tpproc as tp

In [ ]:
archivos = glob.glob('./Train/*/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos)
df_train.head()

In [ ]:
archivos = glob.glob('./Test/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos,test=True)
df_test.head()

## Formateo de Data Frames de trabajo (paso a paso)

In [ ]:
df_train_T = df_train.copy()
df_test_T = df_test.copy()

In [ ]:
from collections import Counter  #regresa un diccionario con conteos

df_train_T['palabras']=df_train_T['texto'].apply(lambda x: x.split())
df_train_T['total']=df_train_T['palabras'].apply(lambda x: len(x))
df_train_T['conteos']=df_train_T['palabras'].apply(lambda x: Counter(x))

df_train_T=df_train_T.sort_values(by="total",ascending=False)
df_train_T.head()
# df_train_T.tail()

In [ ]:
df_train_T['conteos']=df_train_T['conteos'].apply(lambda x:{k: v for k, v in sorted(x.items(), key=lambda item: item[1],reverse=True)})
df_train_T.head()

In [ ]:
df_train_T['topw']=df_train_T['conteos'].apply(lambda x:list(x.keys())[:5])
df_train_T.head()

In [ ]:
top=5
df_train_T['topw']=df_train_T['conteos'].apply(lambda x:list({k: v for k, v in sorted(x.items(), key=lambda item: item[1],reverse=True)}.keys())[:top])
df_train_T.head()

## Todo en 1
### Data Frames de Trabajo
- **Se puede jugar con el parámetro _top_ para aumentar o reducir el vocabulario**
- Se crean diccionarios para indexar las palabras a valores numéricos

In [ ]:
df_train_T = df_train.copy()
df_test_T = df_test.copy()

"""
Cambiar si se desea (y se puede) aumentar el vocabulario
"""
top=5

df_train_T['topw']=df_train_T['texto'].apply(lambda x:[w[0] for w in sorted(Counter(x.split()).items(), key=lambda item: item[1],reverse=True)[:top]])
df_test_T['topw']=df_test_T['texto'].apply(lambda x:[w[0] for w in sorted(Counter(x.split()).items(), key=lambda item: item[1],reverse=True)[:top]])

### Vocabulario de entrenamiento y prueba
vTR={}
vTS={}

# Obtenemos el vocabulario de entrenamiento
i=0
for l in df_train_T.topw.values:
    for w in l:
        if w not in vTR:
            vTR[w]=i
            i+=1
vTR['UNK']=i # Agregamos palabra desconocida 'UNK'

print(len(vTR), 'palabras en vocabulario entrenamiento, incluyendo palabra desconocida (UNK)')

# Sustituimos palabra que no esta en vTR por palabra desconocida 'UNK' en textos de prueba
df_test_T['topw']=df_test_T['topw'].apply(lambda x:[w if w in vTR.keys() else 'UNK' for w in x])

# Obtenemos el vocabulario de prueba
i=0
for l in df_test_T.topw.values:
    for w in l:
        if w not in vTS: 
            vTS[w]=i
            i+=1

print(len(vTS), 'palabras en vocabulario de prueba, incluyendo palabra desconocida (UNK)')

df_tr= df_train_T.filter(['titulo','tipo','autor'], axis=1)
df_ts= df_test_T.filter(['titulo'], axis=1)
for i in range(top):
    df_tr['w{}'.format(i)]=''
for i in range(top):
    df_ts['w{}'.format(i)]=''

for i,row in enumerate(df_train_T.iterrows()):
    l=row[1]['topw']
    for j,w in enumerate(l):
        df_tr.iloc[i,df_tr.columns.get_loc('w{}'.format(j))]=w
for i,row in enumerate(df_test_T.iterrows()):
    l=row[1]['topw']
    for j,w in enumerate(l):
        df_ts.iloc[i,df_ts.columns.get_loc('w{}'.format(j))]=w
   
display(df_tr.head())
display(df_ts.head())


## Preguntas

1. ¿Quién escribió "El caballo y el Lobo"?
2. "La escopeta" ¿es una fábula?
3. ¿Qué probabilidad hay de que "La muerte" haya sido escrita por José Saramago
4. ¿Pudo "La marcha del caracol" haber sido escrita por García Márquez y Fuentes en co-autoría?
5. Dado que "El águila de ala cortada y la zorra" es una fábula, ¿qué probabilidad hay de que haya sido escrita por Jalil Gibrán?
6. ¿Qué proporción hay de minicuentos?
7. ¿Qué cuentos son de Carlos Onetti?
8. ¿Quién dijo: "Desde que abrí los ojos me di cuenta que mi sitio no estaba aquí, donde yo estoy, sino en donde no estoy ni he estado nunca. En alguna parte hay un lugar vacío y ese vacío se llenará de mí y yo me asentaré en ese hueco que insensiblemente rebosará de mí, pleno de mí hasta volverse fuente o surtidor. Y mi vacío, el vacío de mí que soy ahora, se llenará de sí, pleno de sí, pleno de ser hasta los bordes."?
9. ¿Quién es el autor con más cuentos?
10. ¿Qué probablidad hay de quien haya escrito "Retiniblastoma" haya también escrito "El sol y las ranas"?

# Avance de una solución posible

## Modelo:
### Suponiendo independencia condicional de las palabras conociendo el tipo:

<img src="Cuentos-Modelo.jpeg" width=400/> <img src="Cuentos-Modelo-2.jpeg" width=400/>

### $P(A)P(T|A)\prod_{i=1}^N P(W_i|T)$

## Variables

### Conteo de autores y tipos de cuento

In [ ]:
# Ocurrencias de cada autor y conteos de ocurrencias y de número de autores
oc_autor = Counter(df_train_T.autor)
tot_oc_aut = np.sum(list(oc_autor.values()))
tot_autores = len(oc_autor)

print('{0} autores en total: {1}'.format(tot_autores,list(oc_autor.keys())))
print('{} menciones totales'.format(tot_oc_aut))

# Ocurrencias de cada tipo y conteos de ocurrencias y de número de tipos
oc_tipo = Counter(df_train_T.tipo)
tot_oc_tipo = np.sum(list(oc_tipo.values()))
tot_tipos = len(oc_tipo)

print('{0} tipos de cuento en total: {1}'.format(tot_tipos,list(oc_tipo.keys())))
print('{} menciones totales'.format(tot_oc_tipo))

### Definición de variables MIB

#### **Nota Importante**
* Por ahora, el motor de inferencia sólo acepta valores numéricos como valores posibles de una variable.
* Por lo tanto, vamos a crear un mapeo entre los nombres de los autores y un valor entero mediante un diccionario.
* También creamos un mapeo inverso, para poder recuperar las etiquetas a partir de los enteros.
* Haremos lo mismo con los tipos.

In [ ]:
"""
Autor
"""
#diccionario autor->valor
autor_val = dict(zip(oc_autor.keys(),range(len(oc_autor.keys()))))
#diccionario valor->autor
val_autor = dict(zip(range(len(oc_autor.keys())),oc_autor.keys()))
print(autor_val)
print(val_autor)
A=mb.Var('A')
A.set_values(range(len(autor_val)))
print(A.print_var())

"""
Tipo
"""



"""
Palabras
"""
#Palabras, presente:1 ausente:0
#Se crean de manera similar a como se hizo para el filtro SPAM


In [ ]:
print(f'W[{25}]: {W[25]}')

## Identificación de parámetros (estadísticas)